# Imports

In [1]:
import json
import matplotlib.pyplot as plt
import pickle


In [2]:
with open('../../data/template_ids.txt') as json_file:
    templ_ids_dict = json.load(json_file)
    
templ_ids = []

for i in templ_ids_dict:
    templ_ids.append(i)
    
#templ_ids.append('actWhatStart')
    
#templ_ids.append('actWhatEnd')


banned_templates = ['actWhatBetweenWhileEndOneLast',
                    'actWhatBetweenWhileStartOneLast',
                    'actWhatBetweenWhileEndOneFirst',
                    'actWhatBetweenWhileStartOneFirst',
                    'actWhatStart',
                    'actWhatEnd',
                    'actCountChooseMore',
                    'actCountChooseFewer',
                    'actCount'
                    'relFirst',
                    'relLast',
                    'actExists']


for i in banned_templates:
    templ_ids.append(i)
len(templ_ids)

39

In [3]:
infile = open('../../data/video_lengths.pkl', 'rb')
LENGTHS = pickle.load(infile)
infile.close()

In [4]:

infile = open('../../data/stsgs/test_stsgs.pkl', 'rb')
test_stsgs = pickle.load(infile)
infile.close()

In [5]:
infile = open('../../data/stsgs/train_stsgs.pkl', 'rb')
train_stsgs = pickle.load(infile)
infile.close()




In [6]:
#train_path = '../../exports/dataset/balanced/train/'
#train_stsgs = [f for f in listdir(train_path) if isfile(join(train_path, f))]
#train_stsgs = [i[:5] for i in train_stsgs]



In [7]:
with open('../../data/templ2global.txt', 'rb') as f:
    templ2global = json.load(f)

In [8]:
illegal_comp_title = 'illegal_comp'
same_direct_title = 'same_direct'
steps_title = 'steps'
ans_type_title = 'ans_type'
global_title = 'global_vars'
lengths_title = 'lengths'

# General Methods

In [9]:
def getQ(q_id):
    v_id = q_id.split("-")[0]
    return QA[v_id][q_id]

def qaTotal():
    total = 0
    for v_id in QA:
        total = total + len(QA[v_id])
    print("QA Total: ", total)

def printLens(dic, name):
    print()
    print(name)
    total = 0
    for i in dic:
        print(i, len(dic[i]))
        total = total + len(dic[i])
    print("TOTAL: ", total)
    
def pickleDump(var, destination):
    f = open(destination,"wb")
    pickle.dump(var,f)
    f.close() 
    
def pickleLoad(path):
    file = open(path, 'rb')
    info = pickle.load(file)
    file.close()
    return info

    
def dumpMetrics(var, title, d_path, group, balanced):
    
    if type(balanced) == bool:
        print("in dump metrics, balanced should be text")
        return
    path = '../../exports/%s/metrics/%s/%s/%s.pkl' % (d_path, balanced, group, title)
    print("dumping to ", path)
    print()
    pickleDump(var, path)
    
def loadMetrics(title, d_path, group, balanced):
    if type(balanced) == bool:
        print("balanced should be text")
        return
    path = '../../exports/%s/metrics/%s/%s/%s.pkl' % (d_path, balanced, group, title)
    
    return pickleLoad(path)
    

In [10]:
def getVideoIDS(group):
    if group == 'train':
        return list(train_stsgs)
    elif group == 'test':
        return list(test_stsgs)
    
    elif group == 'names':
        return names_stsgs
    
def loadJSON(path):
    with open(path) as json_file:
        QA = json.load(json_file)
    return QA

def getVideoQA(d_path, group, balanced, v_id):
    if type(balanced) == bool: 
        print("balanced should not be bool")
    #if balanced == 'templ':
    
    if group == 'names':
        group = 'test'
    
    QA = loadJSON('../../exports/%s/%s/%s/%s.txt' % (d_path, balanced, group, v_id))
        
   #elif balanced:
   #     QA = loadJSON('../exports/%s/balanced/%s/%s.txt' % (d_path, group, v_id))
# elif not balanced:
   #     QA = loadJSON('../exports/%s/all/%s/%s.txt' % (d_path, group, v_id))
        
    return QA

In [11]:
def getAllOthers(QA, to_del):
    
    for q_id in to_del:
        del QA[q_id]
        
    return list(QA)

In [12]:
def splitQIDS(idx_dic, q_ids_dic):
    split = {
            'less': {},
            'more': {}
        }
    for i in idx_dic:
        split_idx = idx_dic[i]
        for step in q_ids_dic[i]:
            if step <= split_idx:
                cat = 'less'
            else:
                cat = 'more'
                
            for q_id in q_ids_dic[i][step]:
                v_id = q_id[:5]
                if v_id not in split[cat]:
                    split[cat][v_id] = []
                split[cat][v_id].append(q_id)
            
    return split
    

In [13]:
getVideoIDS('test')[0]

'YSKX3'

# Metrics Methods

In [14]:
# Novel Compositions

def getIllegalComp(d_path, group, balanced):
    if type(balanced) == bool:
        print("balanced should be text")
        return
    ids = ['before', 'first', 'longer', 'repetition', 'objrel']

    illegal_comp = {
        'before': [],
        'first': [],
        'longer': [],
        'repetition': [],
        'objrel': []
    }
    v_ids = getVideoIDS(group)
    test_id = 0
    cnt = 0
    comp_by_vid = {}
    for v_id in v_ids:
        comp_by_vid[v_id] = set()
        if cnt == 0:
            test_id = v_id
        if cnt % 1000 == 0:
            print(cnt)
        cnt = cnt + 1
        
        QA = getVideoQA(d_path, group, balanced, v_id)
        for q_id in QA:
            q = QA[q_id]
            metrics = q['metrics']
            nov_comp = metrics['novel_comp']
            for i in range(len(nov_comp)): 
                #if group == 'train':
                if nov_comp[i]:
                    illegal_comp[ids[i]].append(q_id)
            #    if group == 'test':
                    #if not nov_comp[i]:
                        #illegal_comp[ids[i]].append(q_id)
                    
    for glob in illegal_comp:
        print(glob)
        for q_id in illegal_comp[glob]:
            v_id = q_id[:5]
            
            #if group == 'train'
            #if v_id not in comp_by_vid:
            #    comp_by_vid[v_id] = set()
            comp_by_vid[v_id].add(q_id)
    
    print("length of vidoe before: ", len(comp_by_vid[test_id]))
    for i in comp_by_vid[test_id]:
        print(i)

    # right now its collected all the ones that have novel comps, but we want to retain those that don't in test set
    if group == 'test':
        for v_id in comp_by_vid:
            QA = getVideoQA(d_path, group, balanced, v_id)
            

            if v_id == test_id:
                print("length of the whle qa", len(QA))
                
            
            comp_by_vid[v_id] = getAllOthers(QA, comp_by_vid[v_id])
            
            
            for q_id in comp_by_vid[v_id]:
                q = QA[q_id]
                if q['metrics']['novel_comp'].count(True) > 0:
                    print("There is a novel composition!!!")

    print("updated comb bby video", len(comp_by_vid[test_id]))
    
    
    return comp_by_vid, illegal_comp

In [15]:
# Indirect Ref Consistency

# first, by templ/v_id and what asking

def getIndirectRefConsistency(d_path, group, balanced):
    if type(balanced) == bool:
        print("balanced should be text")
        return
    ids = ['object', 'relation', 'action', 'time']

    same_direct = {}

    cnt = 0
    video_ids = getVideoIDS(group)
    for v_id in video_ids:
        if cnt % 100 == 0:
            print(cnt)
        cnt = cnt + 1
        
        QA = getVideoQA(d_path, group, balanced, v_id)
        same_direct[v_id] = {}
        for i in templ_ids:
            same_direct[v_id][i] = {}

        for q_id in QA:
            q = QA[q_id]
            t_id = q['attributes']['type']
            indirects = q['metrics']['indirects']

            direct = q['metrics']['direct_args']

            sd_templ = same_direct[v_id][t_id]

            if direct not in sd_templ:
                sd_templ[direct] = {}
                sd_templ[direct]['all'] = []
                sd_templ[direct]['direct'] = []
                sd_templ[direct]['object'] = []
                sd_templ[direct]['relation'] = []
                sd_templ[direct]['action'] = []
                sd_templ[direct]['time'] = []
                sd_templ[direct]['one'] = {}
                sd_templ[direct]['one']['object'] = []
                sd_templ[direct]['one']['relation'] = []
                sd_templ[direct]['one']['action'] = []
                sd_templ[direct]['one']['time'] = []
                sd_templ[direct]['two'] = []

                
            #print('sddirect', sd_templ[direct])

            sd_templ[direct]['all'].append(q_id)
            
            for idx, i in enumerate(indirects):
                if i:
                    #print(ids[idx])
                    sd_templ[direct][ids[idx]].append(q_id)
            

            num_true = indirects.count(True)

            if num_true == 0:
                sd_templ[direct]['direct'].append(q_id)
            elif num_true == 1:
                chose_something = False
                for i in range(len(indirects)):
                    
                    if indirects[i]:
                        sd_templ[direct]['one'][ids[i]].append(q_id)
                        chose_something = True
                        #print()
                        #print(q_id, ids[i])
                        #print(sd_templ[direct]['one'][ids[i]])
                #if not chose_something:
                    #print(q_id, 'not chose something')
            else:
                sd_templ[direct]['two'].append(q_id)
            
        
        #print()
        #print("FINAL")
        #print('sddirect', sd_templ[direct]))
        #print(same_direct[test_id]['andObjRelExists'])
    return same_direct


In [16]:
# Number of compositional steps
def getNumSteps(d_path, group, balanced):
    if type(balanced) == bool:
        print("balanced should be text")
        return
    steps = {}
    steps_by_templ = {}
    steps_by_glob = {}
    
    num_int = 20

    for i in range(num_int):
        steps[i] = []

    v_ids = getVideoIDS(group)
    count = 0
    for v_id in v_ids:
        if count % 1000 == 0:
            print(count)
        count = count + 1
        QA = getVideoQA(d_path, group, balanced, v_id)
        for q_id in QA:
            q = QA[q_id]
            t = q['attributes']['type']
            g = q['global']
            #q = getQ(q_id)
            q_steps = q['steps']

            
            if t not in steps_by_templ:
                steps_by_templ[t] = {}
                for i in range(num_int):
                    steps_by_templ[t][i] = []
                    
            
            if g not in steps_by_glob:
                steps_by_glob[g] = {}
                for i in range(num_int):
                    steps_by_glob[g][i] = []
                    
                    
            steps[q_steps].append(q_id)
            steps_by_templ[t][q_steps].append(q_id)
            steps_by_glob[g][q_steps].append(q_id)
            
    return steps, steps_by_templ, steps_by_glob


In [17]:
# Answer Type
# Global

def getAnsTypeAndGlobal(d_path, group, balanced):
    if type(balanced) == bool:
        print("balanced should be text")
        return
    answer_type = {
        'binary': [],
        'open': [],
    }


    old_glob = {
        'exists': [],
        'first': [],
        'last': [],
        'what': [],
        'count': [],
        'length': [],
        'time': [],
    }
    
    glob = {
        'obj-rel': [],
        'obj-act': [],
        'rel-act': [],
        'duration-comparison': [],
        'superlative': [],
        'exists': [],
        'count': [],
        'action-recognition': [],
        'sequencing': [],
    }
    
    answer_type = {
        'binary': 0,
        'open': 0,
    }


    old_glob = {
        'exists': 0,
        'first': 0,
        'last': 0,
        'what': 0,
        'count': 0,
        'length': 0,
        'time': 0,
    }
    
    glob = {
        'obj-rel': 0,
        'obj-act': 0,
        'rel-act': 0,
        'duration-comparison': 0,
        'superlative': 0,
        'exists': 0,
        'count': 0,
        'action-recognition': 0,
        'sequencing': 0,
    }
    
    
    templates = {}
    
    v_ids = getVideoIDS(group)
    count = 0
    for v_id in v_ids:
        if count % 1000 == 0:
            print(count)
        count = count + 1
        QA = getVideoQA(d_path, group, balanced, v_id)
        for q_id in QA:
            q = QA[q_id] #getQ(q_id)
            struct = q['attributes']['structural']
            if struct == 'query':
                q_ans_type = 'open'
            else:
                q_ans_type = 'binary'
                
            #answer_type[q_ans_type].append(q_id)
            
            
            answer_type[q_ans_type] += 1
            
            
            
            templ = q['attributes']['type']
            for q_global in templ2global[templ]:
                #glob[q_global].append(q_id)
                glob[q_global] += 1
            if templ not in templates:
                #templates[templ] = []
                templates[templ] = 0
            #templates[templ].append(q_id)
            templates[templ] += 1
            
            #q_ans_type = q['attributes']['ans_type']
            #q_global = q['global']
        
    return answer_type, glob, templates

#answer_type_a, global_a = getAnsTypeAndGlobal(QA_a)
#answer_type_b, global_b = getAnsTypeAndGlobal(QA_b)


In [18]:
# video_lengths

def getLengths(d_path, group, balanced):
    if type(balanced) == bool:
        print("balanced should be text")
        return
    length_gen = {
        'less': [],
        'more': [],
    }

    threshold = 32

    v_ids = getVideoIDS(group)
    count = 0
    for v_id in v_ids:
        if count % 1000 == 0:
            print(count)
        count = count + 1
        QA = getVideoQA(d_path, group, balanced, v_id)
        length = LENGTHS[v_id]
        q_ids = list(QA.keys())
        new_ids = []
        if length <= threshold:
            length_gen['less'].append(v_id)
        else:
            length_gen['more'].append(v_id)

    
    return length_gen


In [19]:
# first, split it into steps dics by global and by template

def addSteps(dic, cat, step):
    if cat not in dic:
        dic[cat] = {}
    if step not in dic[cat]:
        dic[cat][step] = 0
    dic[cat][step] += 1
    
    return dic

def splitCompoStepsByGlobTempl(d_path, group, balanced):
    if type(balanced) == bool: 
        print("invalid balanced")
        return
    by_t = {}
    by_g = {}
    
    v_ids = list(getVideoIDS(group))
    
    for v_id in v_ids:
        QA = getVideoQA(d_path, group, balanced, v_id) 
        
        for q_id in QA:
            q = QA[q_id]
            
            t = q['attributes']['type']
            g = q['global']
            s = q['steps']
            
            by_g = addSteps(by_g, g, s)
            by_t = addSteps(by_t, t, s)
            
    return by_g, by_t
       
def splitSteps(dic, split_idx):
    split = {
        'less': 0,
        'more': 0,
    }
        
    for step in dic:
        if step <= split_idx:
            cat = 'less'
        else:
            cat = 'more'
        split[cat] += dic[step]
        
    return split
        
def findUniversalSplit(dic, split_idx):
    split = {}
    for i in dic:
        split[i] = splitSteps(dic[i], split_idx)
    return split

In [20]:
def findSplit(dic, perc):
    tot = 0
    copy_dic = {}
    for i in dic:
        tot += dic[i]
        copy_dic[i] = dic[i]
        
    
    split_point = int(tot * perc)
    split_idx = 0
    tot = 0
    while tot < split_point:
        min_key = min(copy_dic)
        tot += copy_dic[min_key]
        del copy_dic[min_key]
        split_idx = min_key
        
    return split_idx

def findGlobalSplit(dic, dic_templ, perc):
    # first, find what the split should be, based on the perc
    s = {}
    s_templ = {}
    s_cnt = {}
    for i in dic:
        s_idx = findSplit(dic[i], perc)
        
        s[i] = splitSteps(dic[i], s_idx)
        s_cnt[i] = s_idx
        
    if dic_templ != None:
        for t in dic_templ:
            glob = t_to_g[t]
            idx = s_cnt[glob]
            s_templ[t] = splitSteps(dic_templ[t], idx)
        
    return s_cnt, s, s_templ



# Generate Metrics

In [21]:
d_path = 'dataset'

In [22]:
illegal_comp_title = 'illegal_comp'
illegal_comp_by_type_title = 'illegal_comp_by_typ'
same_direct_title = 'same_direct'
steps_title = 'steps'
steps_by_templ_title = 'steps_by_templ'
ans_type_title = 'ans_type'
global_title = 'global_vars'
lengths_title = 'lengths'
templ_title = 'templates'

In [23]:
d_path = 'dataset'
balanced = 'balanced'
group = 'train'
save = True

il_comp = False
step = False
AGT = True
lens = False
indir = False

if il_comp: # CHECKED
    print("Generate Illegal comp")
    illegal_comp_by_vid_to_del, illegal_comp_by_type = getIllegalComp(d_path, group, balanced)
    if save:
        dumpMetrics(illegal_comp_by_vid_to_del, illegal_comp_title, d_path, group, balanced)
        dumpMetrics(illegal_comp_by_type, illegal_comp_by_type_title, d_path, group, balanced)

if step: # CHECKED splitting by templ  
    print("Generate Steps")
    print("Split comp by glob")
    g, t = splitCompoStepsByGlobTempl(d_path,  group, balanced)
    print("Find global split")
    sp_t_idx, sp_t, _ = findGlobalSplit(t, None, .5)
    print("Find number of steps")
    steps_b, steps_b_templ, steps_b_glob = getNumSteps(d_path, group, balanced)
    print("split Q ids")
    steps_templ_sp = splitQIDS(sp_t_idx, steps_b_templ)
    if save:
        dumpMetrics(steps_templ_sp, steps_by_templ_title, d_path, group, balanced)
        #dumpMetrics(steps_tr_b, steps_title, d_path, group, balanced)
    
if AGT: # CHECKED
    print("Generate ans/glob/templ")    
    ans_type_tr_b, global_tr_b, templ_tr_b = getAnsTypeAndGlobal(d_path, group, balanced)
    if save:
        dumpMetrics(ans_type_tr_b, ans_type_title, d_path, group, balanced)
        dumpMetrics(global_tr_b, global_title, d_path, group, balanced)
        dumpMetrics(templ_tr_b, templ_title, d_path, group, balanced)
    
if lens: # CHECKED
    print("Generate Lengths")
    lengths_tr_b = getLengths(d_path, group, balanced)
    if save:
        dumpMetrics(lengths_tr_b, lengths_title, d_path, group, balanced)

if indir:
    print("Generate Indirect ref consistency")
    same_direct_tr_b = getIndirectRefConsistency(d_path, group, balanced)
    if save:
        dumpMetrics(same_direct_tr_b, same_direct_title, d_path, group, balanced)

Generate ans/glob/templ
0
1000
2000
3000
4000
5000
6000
7000
dumping to  ../../exports/dataset/metrics/balanced/train/ans_type.pkl

dumping to  ../../exports/dataset/metrics/balanced/train/global_vars.pkl

dumping to  ../../exports/dataset/metrics/balanced/train/templates.pkl



In [24]:
d_path = 'dataset'
balanced = 'balanced'
group = 'test'
save = True

il_comp = False
step = False
AGT = True
lens = False
indir = False

if il_comp: # CHECKED
    print("Generate Illegal comp")
    illegal_comp_by_vid_to_del, illegal_comp_by_type = getIllegalComp(d_path, group, balanced)
    if save:
        dumpMetrics(illegal_comp_by_vid_to_del, illegal_comp_title, d_path, group, balanced)
        dumpMetrics(illegal_comp_by_type, illegal_comp_by_type_title, d_path, group, balanced)

if step: # CHECKED splitting by templ  
    print("Generate Steps")
    print("Split comp by glob")
    g, t = splitCompoStepsByGlobTempl(d_path,  group, balanced)
    print("Find global split")
    sp_t_idx, sp_t, _ = findGlobalSplit(t, None, .5)
    print("Find number of steps")
    steps_b, steps_b_templ, steps_b_glob = getNumSteps(d_path, group, balanced)
    print("split Q ids")
    steps_templ_sp = splitQIDS(sp_t_idx, steps_b_templ)
    if save:
        dumpMetrics(steps_templ_sp, steps_by_templ_title, d_path, group, balanced)
        #dumpMetrics(steps_tr_b, steps_title, d_path, group, balanced)
    
if AGT: # CHECKED
    print("Generate ans/glob/templ")    
    ans_type_tr_b, global_tr_b, templ_tr_b = getAnsTypeAndGlobal(d_path, group, balanced)
    if save:
        dumpMetrics(ans_type_tr_b, ans_type_title, d_path, group, balanced)
        dumpMetrics(global_tr_b, global_title, d_path, group, balanced)
        dumpMetrics(templ_tr_b, templ_title, d_path, group, balanced)
    
if lens: # CHECKED
    print("Generate Lengths")
    lengths_tr_b = getLengths(d_path, group, balanced)
    if save:
        dumpMetrics(lengths_tr_b, lengths_title, d_path, group, balanced)

if indir:
    print("Generate Indirect ref consistency")
    same_direct_tr_b = getIndirectRefConsistency(d_path, group, balanced)
    if save:
        dumpMetrics(same_direct_tr_b, same_direct_title, d_path, group, balanced)

Generate ans/glob/templ
0
1000
dumping to  ../../exports/dataset/metrics/balanced/test/ans_type.pkl

dumping to  ../../exports/dataset/metrics/balanced/test/global_vars.pkl

dumping to  ../../exports/dataset/metrics/balanced/test/templates.pkl



In [25]:
d_path = 'dataset'
balanced = 'all'
group = 'train'
save = True

il_comp = True
step = True
AGT = False
lens = False
indir = False

if il_comp: # CHECKED
    print("Generate Illegal comp")
    illegal_comp_by_vid_to_del, illegal_comp_by_type = getIllegalComp(d_path, group, balanced)
    if save:
        dumpMetrics(illegal_comp_by_vid_to_del, illegal_comp_title, d_path, group, balanced)
        dumpMetrics(illegal_comp_by_type, illegal_comp_by_type_title, d_path, group, balanced)

if step: # CHECKED splitting by templ  
    print("Generate Steps")
    print("Split comp by glob")
    g, t = splitCompoStepsByGlobTempl(d_path,  group, balanced)
    print("Find global split")
    sp_t_idx, sp_t, _ = findGlobalSplit(t, None, .5)
    print("Find number of steps")
    steps_b, steps_b_templ, steps_b_glob = getNumSteps(d_path, group, balanced)
    print("split Q ids")
    steps_templ_sp = splitQIDS(sp_t_idx, steps_b_templ)
    if save:
        dumpMetrics(steps_templ_sp, steps_by_templ_title, d_path, group, balanced)
        #dumpMetrics(steps_tr_b, steps_title, d_path, group, balanced)
    
if AGT: # CHECKED
    print("Generate ans/glob/templ")    
    ans_type_tr_b, global_tr_b, templ_tr_b = getAnsTypeAndGlobal(d_path, group, balanced)
    if save:
        dumpMetrics(ans_type_tr_b, ans_type_title, d_path, group, balanced)
        dumpMetrics(global_tr_b, global_title, d_path, group, balanced)
        dumpMetrics(templ_tr_b, templ_title, d_path, group, balanced)
    
if lens: # CHECKED
    print("Generate Lengths")
    lengths_tr_b = getLengths(d_path, group, balanced)
    if save:
        dumpMetrics(lengths_tr_b, lengths_title, d_path, group, balanced)

if indir:
    print("Generate Indirect ref consistency")
    same_direct_tr_b = getIndirectRefConsistency(d_path, group, balanced)
    if save:
        dumpMetrics(same_direct_tr_b, same_direct_title, d_path, group, balanced)

Generate Illegal comp
0


FileNotFoundError: [Errno 2] No such file or directory: '../../exports/dataset/all/train/46GP8.txt'

In [ ]:
d_path = 'dataset'
balanced = 'all'
group = 'test'
save = True

il_comp = False
step = False
AGT = False
lens = False
indir = True

if il_comp: # CHECKED
    print("Generate Illegal comp")
    illegal_comp_by_vid_to_del, illegal_comp_by_type = getIllegalComp(d_path, group, balanced)
    if save:
        dumpMetrics(illegal_comp_by_vid_to_del, illegal_comp_title, d_path, group, balanced)
        dumpMetrics(illegal_comp_by_type, illegal_comp_by_type_title, d_path, group, balanced)

if step: # CHECKED splitting by templ  
    print("Generate Steps")
    print("Split comp by glob")
    g, t = splitCompoStepsByGlobTempl(d_path,  group, balanced)
    print("Find global split")
    sp_t_idx, sp_t, _ = findGlobalSplit(t, None, .5)
    print("Find number of steps")
    steps_b, steps_b_templ, steps_b_glob = getNumSteps(d_path, group, balanced)
    print("split Q ids")
    steps_templ_sp = splitQIDS(sp_t_idx, steps_b_templ)
    if save:
        dumpMetrics(steps_templ_sp, steps_by_templ_title, d_path, group, balanced)
        #dumpMetrics(steps_tr_b, steps_title, d_path, group, balanced)
    
if AGT: # CHECKED
    print("Generate ans/glob/templ")    
    ans_type_tr_b, global_tr_b, templ_tr_b = getAnsTypeAndGlobal(d_path, group, balanced)
    if save:
        dumpMetrics(ans_type_tr_b, ans_type_title, d_path, group, balanced)
        dumpMetrics(global_tr_b, global_title, d_path, group, balanced)
        dumpMetrics(templ_tr_b, templ_title, d_path, group, balanced)
    
if lens: # CHECKED
    print("Generate Lengths")
    lengths_tr_b = getLengths(d_path, group, balanced)
    if save:
        dumpMetrics(lengths_tr_b, lengths_title, d_path, group, balanced)

if indir:
    print("Generate Indirect ref consistency")
    same_direct_tr_b = getIndirectRefConsistency(d_path, group, balanced)
    if save:
        dumpMetrics(same_direct_tr_b, same_direct_title, d_path, group, balanced)

# Balanced - Train

In [ ]:
dumpMetrics(templ_tr_b, templ_title, d_path, group, balanced)

# Does temporal Change answer

In [ ]:
# Indirect Ref Consistency

# first, by templ/v_id and what asking

def getIndirectRefConsistencyTime(d_path, group, balanced):
    if type(balanced) == bool:
        print("balanced should be text")
        return
    ids = ['object', 'relation', 'action', 'time']

    same_direct = {}
    comp = {}

    cnt = 0
    video_ids = getVideoIDS(group)
    for v_id in video_ids:
        if cnt % 100 == 0:
            print(cnt)
        cnt = cnt + 1
        comp[v_id] = {}
        
        QA = getVideoQA(d_path, group, balanced, v_id)
        same_direct[v_id] = {}
        for i in templ_ids:
            same_direct[v_id][i] = {}

        for q_id in QA:
            q = QA[q_id]
            t_id = q['attributes']['type']
            indirects = q['metrics']['indirects']

            direct = q['metrics']['direct_args']

            sd_templ = same_direct[v_id][t_id]

            if direct not in sd_templ:
                sd_templ[direct] = {}
                sd_templ[direct]['all'] = []
                sd_templ[direct]['no_time'] = []
                sd_templ[direct]['time'] = []

                

            sd_templ[direct]['all'].append(q_id)
            
            if indirects[3]:
                sd_templ[direct]['time'].append(q_id)
            else:
                sd_templ[direct]['no_time'].append(q_id)

        
        ### think this may have only done stuff for the last sd_templ?
        #for direct in sd_templ:
        #    thing = sd_templ[direct]
        
        for t_id in same_direct[v_id]:
            for direct in same_direct[v_id][t_id]:
                thing = same_direct[v_id][t_id][direct]
                no_time = thing['no_time']

                if len(no_time) > 0:
                    no_time_ans = QA[no_time[0]]['answer']
                else:
                    no_time_ans = "None without time"

                time = thing['time']
                for q_id in time:
                    q_ans = QA[q_id]['answer']
                    if no_time_ans == "None without time":
                        #no_direct.append(q_id)
                        comp[v_id][q_id] = 'no_direct'
                    elif no_time_ans == q_ans:
                        #same_pairs.append((direct[0], q_id))
                        comp[v_id][q_id] = 'same_ans'
                    else:
                        #diff_pairs.append((direct[0], q_id))
                        comp[v_id][q_id] = 'diff_ans'

                for q_id in thing['no_time']:
                    q_ans = QA[q_id]['answer']
                    comp[v_id][q_id] = 'no_temporal'
                    #num_true = indirects.count(True)

                #if num_true == 0:
                #    sd_templ[direct]['direct'].append(q_id)
                #elif num_true == 1:
                #    chose_something = False
                #    for i in range(len(indirects)):

                #        if indirects[i]:
                #            sd_templ[direct]['one'][ids[i]].append(q_id)
                #            chose_something = True
                            #print()
                            #print(q_id, ids[i])
                            #print(sd_templ[direct]['one'][ids[i]])
                    #if not chose_something:
                        #print(q_id, 'not chose something')
                #else:
                #    sd_templ[direct]['two'].append(q_id)
    print("Saving temporal change ans in ../exports/%s/metrics/%s/%s/temporalChangeAns.txt" % (d_path, balanced, group))
    with open('../exports/%s/metrics/%s/%s/temporalChangeAns.txt' % (d_path, balanced, group), 'w+') as f:
        json.dump(comp, f)
    f.close()
    
        
        
    return same_direct, comp


In [ ]:
# probably don't need mult now


def doesTempLocChangeAnswer(d_path, balanced, group, same_direct): 
    #diff_pairs = []
    #same_pairs = []
    #no_direct = []
    
    #if adding:
    if same_direct is None:
        with open('../exports/%s/metrics/%s/%s/temporalChangeAns.txt' % (d_path, balanced, group), 'rb') as f:
            same_direct = json.load(f)
        f.close()
    #else:
    comp = {}
    
    
    for i, v_id in enumerate(same_direct): 
        if i % 2 == 0:
            print(i)
        #with open('../exports/%s/%s/%s/%s.txt' % (d_path, balanced, group, v_id)) as json_file:
       #         QA = json.load(json_file)

        QA = getVideoQA(d_path, group, balanced, v_id)
        #comp[v_id] = {}
        #if adding:
        #    video_mapping = templ_change_ans[v_id]
        #    cnt = len(video_mapping)
        
        if v_id not in comp: 
            comp[v_id] = {}
            
        for t_id in same_direct[v_id]:
            #print(" "*4, t_id)
            for local in same_direct[v_id][t_id]:
                print(same_direct[v_id][t_id])
                thing = same_direct[v_id][t_id][local]
                
                no_time = thing['no_time']

                if len(no_time) > 0:
                    no_time_ans = QA[no_time[0]]['answer']
                else:
                    no_time_ans = "None without time"
                    
                    
                time = thing['time']

                for q_id in time:
                    #print(" "*12, QA[q_id]['answer'], QA[q_id]['question'])
                    #if q_id in 
                    q_ans = QA[q_id]['answer']
                    if no_time_ans == "None without time":
                        #no_direct.append(q_id)
                        comp[v_id][q_id] = 'no_direct'
                    elif no_time_ans == q_ans:
                        #same_pairs.append((direct[0], q_id))
                        comp[v_id][q_id] = 'same_ans'
                    else:
                        #diff_pairs.append((direct[0], q_id))
                        comp[v_id][q_id] = 'diff_ans'
                        
                for q_id in thing['no_time']:
                    q_ans = QA[q_id]['answer']
                    comp[v_id][q_id] = 'no_temporal'

        #if cnt % 1000 == 0:
        #    print(cnt)
        cnt = cnt + 1
        
    #with open('../exports/dataset/metrics/%s/%s/diff_pairs_one.txt' % (balanced, group), 'w+') as f:
    #    json.dump(diff_pairs, f)
    #f.close()
    #with open('../exports/dataset/metrics/%s/%s/same_pairs_one.txt' % (balanced, group), 'w+') as f:
    #    json.dump(same_pairs, f)
    #f.close()
    #with open('../exports/dataset/metrics/%s/%s/no_direct_one.txt' % (balanced, group), 'w+') as f:
    #    json.dump(no_direct, f)
    #f.close()
    print("Saving temporal change ans in ../exports/%s/metrics/%s/%s/temporalChangeAns.txt" % (d_path, balanced, group))
    with open('../exports/%s/metrics/%s/%s/temporalChangeAns.txt' % (d_path, balanced, group), 'w+') as f:
        json.dump(comp, f)
    f.close()
    
        
    return comp


#same_direct_te = getIndirectRefConsistencyTime(d_path, 'test', 'all')
#comp_test = doesTempLocChangeAnswer(d_path, "all", "test", same_direct_te)
#diff_pairs_train, same_pairs_train, no_direct_train, comp_test = doesTempLocChangeAnswer(d_path, "all", "train")

In [ ]:
templ_ids

In [ ]:
group = 'train'
balanced = 'all'
same_direct_tr = getIndirectRefConsistencyTime(d_path, group, balanced)

In [ ]:
with open('../exports/dataset/metrics/all/train/temporalChangeAns.txt', 'rb') as f:
    same_direct_train = json.load(f)
f.close()

In [ ]:
x = set()
for i in same_direct_train:
    #print(i, same_direct_train[i])
    for j in same_direct_train[i]:
        x.add(same_direct_train[i][j])
    break
x

In [ ]:
comp_train = doesTempLocChangeAnswer(d_path, "all", "train", None)

In [ ]:
same_direct_te = getIndirectRefConsistencyTime(d_path, 'test', 'all')

In [ ]:
# Novel Compositions

def getIllegalCompOLD(d_path, group, balanced):
    if type(balanced) == bool:
        print("balanced should be text")
        return
    ids = ['before', 'first', 'longer', 'repetition', 'objrel']

    illegal_comp = {
        'before': [],
        'first': [],
        'longer': [],
        'repetition': [],
        'objrel': []
    }
    v_ids = getVideoIDS(group)
    test_id = 0
    cnt = 0
    for v_id in v_ids:
        if cnt == 0:
            test_id = v_id
        if cnt % 1000 == 0:
            print(cnt)
        cnt = cnt + 1
        
        QA = getVideoQA(d_path, group, balanced, v_id)
        for q_id in QA:
            q = QA[q_id]
            metrics = q['metrics']
            nov_comp = metrics['novel_comp']
            for i in range(len(nov_comp)): 
                #if group == 'train':
                if nov_comp[i]:
                    illegal_comp[ids[i]].append(q_id)
            #    if group == 'test':
                    #if not nov_comp[i]:
                        #illegal_comp[ids[i]].append(q_id)
    
    comp_by_vid = {}  
    for glob in illegal_comp:
        print(glob)
        for q_id in illegal_comp[glob]:
            v_id = q_id[:5]
            
            if group == 'train'
            if v_id not in comp_by_vid:
                comp_by_vid[v_id] = set()
            comp_by_vid[v_id].add(q_id)
    
    print("length of vidoe before: ", len(comp_by_vid[test_id]))
    for i in comp_by_vid[test_id]:
        print(i)

    # right now its collected all the ones that have novel comps, but we want to retain those that don't in test set
    if group == 'test':
        for v_id in comp_by_vid:
            QA = getVideoQA(d_path, group, balanced, v_id)

            if v_id == test_id:
                print("length of the whle qa", len(QA))
                
            
            comp_by_vid[v_id] = getAllOthers(QA, comp_by_vid[v_id])

    print("updated comb bby video", len(comp_by_vid[test_id]))
    
    
    return comp_by_vid, illegal_comp

In [ ]:
for i in same_direct_te:
    print(i, same_direct_te[i])
    break

In [ ]:
comp_test = doesTempLocChangeAnswer(d_path, "all", "test", None)# same_direct_te)

In [ ]:
same_direct_te = getIndirectRefConsistencyTime(d_path, 'test', 'all')

In [ ]:
comp_test = doesTempLocChangeAnswer(d_path, "all", "names", None)# same_direct_te)

In [ ]:
d_path = 'dataset'
balanced = 'all'
group = 'test'
with open('../exports/%s/metrics/%s/%s/temporalChangeAns.txt' % (d_path, balanced, group), 'rb') as f:
    same_direct = json.load(f)
f.close()

In [ ]:
for i in list(same_direct)[:10]:
    QA = getVideoQA(d_path, group, balanced, i)
    QA_b = getVideoQA('dataset CVPR submission', 'test', 'balanced', i)
    print(len(same_direct[i]), len(QA_b), len(QA))

In [ ]:
for i in same_direct:
    print(i)
    for j in same_direct[i]:
        print("    ", j)
    
    break

In [ ]:
len(QA)

# Split by template or by global

In [ ]:
t_to_g = {'objExists': 'what',
 'relExists': 'exists',
 'actExists': 'exists',
 'objWhat': 'what',
 'objFirst': 'first',
 'objFirstVerify': 'first',
 'objLast': 'last',
 'actLengthLongerChoose': 'length',
 'actLengthShorterChoose': 'length',
 'relTime': 'time',
 'objTime': 'time',
 'objWhatGeneral': 'what',
 'actCountChooseMore': 'count',
 'actCountChooseFewer': 'count',
 'objRelExists': 'exists',
 'andObjRelExists': 'exists',
 'xorObjRelExists': 'exists',
 'objWhatChoose': 'what',
 'objFirstChoose': 'first',
 'objLastChoose': 'last',
 'objLastVerify': 'last',
 'actCount': 'count',
 'actLengthLongerVerify': 'length',
 'actLengthShorterVerify': 'length',
 'actTime': 'time',
 'relLast': 'last',
 'relFirst': 'first'}

# Old Stuff

# Finding Pairs with same answer after localize

In [ ]:

def doesTempLocChangeAnswerOLD(d_path, balanced, group, same_direct): 
    diff_pairs = []
    same_pairs = []
    no_direct = []
    
    comp = {}

    cnt = 0
    for i, v_id in enumerate(same_direct): 
        if i % 1000 == 0:
            print(i)
            
        if group == 'names'
        with open('../exports/%s/%s/%s/%s.txt' % (d_path, balanced, group, v_id)) as json_file:
                QA = json.load(json_file)

        comp[v_id] = {}
        for t_id in same_direct[v_id]:
            #print(" "*4, t_id)
            for direct in same_direct[v_id][t_id]:
                thing = same_direct[v_id][t_id][direct]
                direct = thing['direct']

                if len(direct) > 0:
                    #print(" "*8, QA[direct[0]]['answer'], QA[direct[0]]['question'])
                    direct_ans = QA[direct[0]]['answer']
                else:
                    direct_ans = "No Direct"
                time = thing['one']['time']

                for q_id in time:
                    #print(" "*12, QA[q_id]['answer'], QA[q_id]['question'])
                    q_ans = QA[q_id]['answer']
                    if direct_ans == "No Direct":
                        no_direct.append(q_id)
                        comp[v_id][q_id] = 'no_direct'
                    elif direct_ans == q_ans:
                        same_pairs.append((direct[0], q_id))
                        comp[v_id][q_id] = 'same_ans'
                    else:
                        diff_pairs.append((direct[0], q_id))
                        comp[v_id][q_id] = 'diff_ans'

        if cnt % 1000 == 0:
            print(cnt)
        cnt = cnt + 1
        
    with open('../exports/dataset/metrics/%s/%s/diff_pairs_one.txt' % (balanced, group), 'w+') as f:
        json.dump(diff_pairs, f)
    f.close()
    with open('../exports/dataset/metrics/%s/%s/same_pairs_one.txt' % (balanced, group), 'w+') as f:
        json.dump(same_pairs, f)
    f.close()
    with open('../exports/dataset/metrics/%s/%s/no_direct_one.txt' % (balanced, group), 'w+') as f:
        json.dump(no_direct, f)
    f.close()
    with open('../exports/dataset/metrics/%s/%s/comp_one.txt' % (balanced, group), 'w+') as f:
        json.dump(comp, f)
    f.close()
    
        
    return diff_pairs, same_pairs, no_direct, comp

#d_path = 'dataset before 11-4'

#same_direct_te = getIndirectRefConsistencyTime(d_path, 'test', 'all')
#diff_pairs_test, same_pairs_test, no_direct_test, comp_test = doesTempLocChangeAnswer(d_path, "all", "test", same_direct_te)
#diff_pairs_train, same_pairs_train, no_direct_train, comp_test = doesTempLocChangeAnswer(d_path, "all", "train")

In [ ]:
# Testing when have multiple. 

def doesTempLocChangeAnswerMultIndirectOLD(d_path, balanced, group, same_direct): 
    diff_pairs = []
    same_pairs = []
    no_direct = []

    comp = {}
    cnt = 0
    for v_id in same_direct: 
        with open('../exports/%s/%s/%s/%s.txt' % (d_path, balanced, group, v_id)) as json_file:
                QA = json.load(json_file)

        comp[v_id] = {}
        for t_id in same_direct[v_id]:
            #print(" "*4, t_id)
            for direct in same_direct[v_id][t_id]:
                thing = same_direct[v_id][t_id][direct]
                direct = thing['direct']

                if len(direct) > 0:
                    #print(" "*8, QA[direct[0]]['answer'], QA[direct[0]]['question'])
                    direct_ans = QA[direct[0]]['answer']
                else:
                    direct_ans = "No Direct"
                mult = thing['two']
                #time = thing['one']['time']

                for q_id in mult:
                    #print(" "*12, QA[q_id]['answer'], QA[q_id]['question'])
                    q = QA[q_id]
                    
                    indirects = q['metrics']['indirects']
                    if not indirects[3]:
                        continue
                        
                    q_ans = q['answer']
                    if direct_ans == "No Direct":
                        no_direct.append(q_id)
                        comp[v_id][q_id] = 'no_direct'
                    elif direct_ans == q_ans:
                        same_pairs.append((direct[0], q_id))
                        comp[v_id][q_id] = 'same_ans'
                    else:
                        diff_pairs.append((direct[0], q_id))
                        comp[v_id][q_id] = 'diff_ans'

        if cnt % 1000 == 0:
            print(cnt)
        cnt = cnt + 1
        
    
    with open('../exports/dataset/metrics/%s/%s/diff_pairs_mult.txt' % (balanced, group), 'w+') as f:
        json.dump(diff_pairs, f)
    f.close()
    with open('../exports/dataset/metrics/%s/%s/same_pairs_mult.txt' % (balanced, group), 'w+') as f:
        json.dump(same_pairs, f)
    f.close()
    with open('../exports/dataset/metrics/%s/%s/no_direct_mult.txt' % (balanced, group), 'w+') as f:
        json.dump(no_direct, f)
    f.close()
    with open('../exports/dataset/metrics/%s/%s/comp_mult.txt' % (balanced, group), 'w+') as f:
        json.dump(comp, f)
    f.close()
    return diff_pairs, same_pairs, no_direct, comp


#diff_pairs_test_mult, same_pairs_test_mult, no_direct_test_mult, comp_test_mult = doesTempLocChangeAnswerMultIndirect(d_path, "all", "test", same_direct_te)
#diff_pairs_train_mult, same_pairs_train_mult, no_direct_train_mult, comp_train_mult = doesTempLocChangeAnswerMultIndirect("all", "train", same_direct_tr_b)

In [ ]:
no_temp = 0
same = 0
diff = 0
no_dir = 0

for v_id in comp_test:
    for q_id in comp_test[v_id]:
        val = comp_test[v_id][q_id]
        if val == 'same_ans':
            same += 1
        elif val == 'no_temporal':
            no_temp += 1
        elif val == 'diff_ans':
            diff += 1
        elif val == 'no_direct':
            no_dir +=1
        else:
            print("Invalid value", val)

print("No temporal loc in ", no_temp)
print("Same ans as direct in ", same)
print("Different ans than direct in", diff)
print("no direct in ", no_dir)

In [ ]:
with open('../exports/%s/%s/%s/%s.txt' % ('dataset before 11-4', 'all', 'test', 'YSKX3')) as json_file:
    QA = json.load(json_file)

c = comp_test['YSKX3']
cm = comp_test_mult['YSKX3']

for q_id in QA:
    if q_id in c:
        tp = c[q_id]
    elif q_id in cm:
        tp = cm[q_id]
    else:
        tp = 'direct'
        
    print(q_id, tp, QA[q_id]['question'])

In [ ]:
# so if its not in it , its direect

# well that's how it should be but its not - so double check


# OK SO THEORETICALLY - I Could instead split it by time, not time. and then see. would be easy
# can just make another funciton... and then don't run into this no direct problem ( i think) or would be a lot smaller
# and hen just make everthing no_time

In [ ]:
dic = same_direct_te['YSKX3']['andObjRelExists']['v008-o4-o26'] #same_direct_te['YSKX3']['andObjRelExists']['r22-o4-o26']
for i in dic: #[t_id][direct]
    print(i)
    for j in dic[i]:
        print("   ", j)
        if i == 'one':
            for k in dic[i][j]:
                print("     ", k)

In [ ]:
small = getIndirectRefConsistencyTime(d_path, 'test', 'all')


if False:
    print("test_one", len(diff_pairs_test), len(same_pairs_test), len(no_direct_test))
    print("test_mult", len(diff_pairs_test_mult), len(same_pairs_test_mult), len(no_direct_test_mult))
    #print("train", len(diff_pairs_train), len(same_pairs_train), len(no_direct_train))
    #
    # NOTE! This doesnt involve ones with two where the temporal localization does nothing, but im not sure how to do that. Maybe if different from direct and it has a temporal localizor

    same_pairs_test_all = same_pairs_test + same_pairs_test_mult
    print(len(same_pairs_test_all))


if False:
    with open("../AMT/easyturk/easyturk/data/same_pairs_test.txt", 'w+') as outfile:
        json.dump(same_pairs_test_all, outfile)


if False: 
    for i in diff_pairs:
        with open('../exports/dataset/test/%s.txt' % i[0][:5]) as json_file:
                QA = json.load(json_file)
        print()
        print(QA[i[0]]['answer'], QA[i[0]]['question'])
        print(QA[i[1]]['answer'], QA[i[1]]['question'])

# Generate Metrics

In [ ]:
sp_g_tr_idx

In [ ]:
for i in steps_glob_sp_te['less']['T5ECU']:
    print(i)

In [ ]:

g_tr, t_tr = splitCompoStepsByGlobTempl('dataset', 'train', 'templ_balanced')
g_te, t_te = splitCompoStepsByGlobTempl('dataset', 'test', 'templ_balanced')
sp_g_tr_idx, sp_g_tr, sp_g_tr_templ = findGlobalSplit(g_tr, t_tr, .5)
sp_g_te_idx, sp_g_te, sp_g_te_templ = findGlobalSplit(g_te, t_te, .5)
sp_t_tr_idx, sp_t_tr, _ = findGlobalSplit(t_tr, None, .5)
sp_t_te_idx, sp_t_te, _ = findGlobalSplit(t_te, None, .5)

steps_glob_sp_tr = splitQIDS(sp_g_tr_idx, steps_tr_b_glob)
steps_glob_sp_te = splitQIDS(sp_g_te_idx, steps_te_b_glob)
steps_templ_sp_tr = splitQIDS(sp_t_tr_idx, steps_tr_b_templ)
steps_templ_sp_te = splitQIDS(sp_t_te_idx, steps_te_b_templ)

#dumpMetrics(steps_glob_sp_tr, "steps_by_glob", d_path, 'train', balanced)
#dumpMetrics(steps_glob_sp_te, "steps_by_glob", d_path, 'test', balanced)
dumpMetrics(steps_templ_sp_tr, "steps_by_templ", d_path, 'train', balanced)
dumpMetrics(steps_templ_sp_te, "steps_by_templ", d_path, 'test', balanced)

# Get Numbers on Metrics

In [ ]:
d_path = 'dataset'
illegal_comp_te_b = loadMetrics(illegal_comp_title, d_path, 'test', balanced)
illegal_comp_tr_b = loadMetrics(illegal_comp_title, d_path, 'train', balanced)
#illegal_comp_te_a = loadMetrics(illegal_comp_title, d_path, 'test', balanced=False)
#illegal_comp_tr_a = loadMetrics(illegal_comp_title, d_path, 'train', balanced=False)

In [ ]:
for glob in illegal_comp_tr_b:
    for q_id in illegal_comp_tr_b[glob]:
        if q_id[:5] == '46GP8':
            print(q_id)

In [ ]:
print()
print("TRAIN all")
#for i in illegal_comp_tr_a:
#    print(i, len(illegal_comp_tr_a[i]))
print()
print("TEST all")
#for i in illegal_comp_te_a:
#    print(i, len(illegal_comp_te_a[i]))
print()
print("TRAIN balanced")
for i in illegal_comp_tr_b:
    print(i, len(illegal_comp_tr_b[i]))
print()
print("TEST balanced")
for i in illegal_comp_te_b:
    print(i, len(illegal_comp_te_b[i]), illegal_comp_te_b[i][0])

In [ ]:
type(illegal_comp_te_b['before'][0])

In [ ]:
#dumpMetrics(comp_by_vid_train, illegal_comp_title + "-by-vid", d_path, 'train', balanced)
#dumpMetrics(comp_by_vid_test, illegal_comp_title + "-by-vid", d_path, 'test', balanced)

In [ ]:
total = 0
for v in comp_by_vid_test:
    total = total + len(comp_by_vid_test[v])
    
total

In [ ]:
comp_by_vid['YSKX3'], len(comp_by_vid['YSKX3'])

In [ ]:
IDX = pickleLoad('../data/IDX.pkl')
PP = pickleLoad('../data/pres_part.pkl')
PRES = pickleLoad('../data/present.pkl')
PAST = pickleLoad('../data/past.pkl')
import grammar as g

In [ ]:
BEFORE_COMP = [
        IDX["standing up"],
        IDX["playing with a phone"],
        IDX["throwing a broom somewhere"],
        #IDX["holding a blanket"],
    ]

FIRST_COMP = [
        IDX['looking_at'],
        IDX['behind'],
        IDX['holding'],
    ]

LONGER_COMP = [
        IDX["standing up"],
        IDX["playing with a phone"],
        IDX["throwing a broom somewhere"],
        #IDX["holding a blanket"],
    ]

REPETITION_COMP = [
        IDX['taking a cup from somewhere'],
        IDX['putting some food somewhere'],
        #IDX["holding a blanket"],
    ]

# THese cannot be in actions
OBJREL_COMP = [
        (IDX['table'], IDX['wiping']), #TODO: these need to be changed b/c touching for a lot of other things. needs to be more distinct verbs
        (IDX['cup'], IDX['wiping']),
        (IDX['table'], IDX['beneath']),
        (IDX['cup'], IDX['beneath']),
        (IDX['food'], IDX['in front of']),
        (IDX['book'], IDX['carrying']),
        (IDX['chair'], IDX['leaning on']),
        #(IDX['table'], IDX['touching']), #TODO: these need to be changed b/c touching for a lot of other things. needs to be more distinct verbs
        #(IDX['food'], IDX['touching']),
        #(IDX['table'], IDX['beneath']),
        #(IDX['food'], IDX['beneath']),
       # (IDX['blanket'], IDX['holding']),
       # (IDX['blanket'], IDX['carrying']),
    ]

NOVEL_COMPS = {
    'before': BEFORE_COMP,
    'first': FIRST_COMP,
    'longer': LONGER_COMP,
    'repetition': REPETITION_COMP,
    'objrel': OBJREL_COMP
}

def substringSearch(q_id, q, ncomp):
    question = q['question']
    cats = set()
    if ncomp == 'before': 
        for act in NOVEL_COMPS[ncomp]:
            phrases = [
                "before starting to %s" % PRES[act],
                "before %s" % PP[act]
            ]
            for phrase in phrases:
                if question.find(phrase) != -1:
                    cats.add(act)
       # "before starting to %s" % (PRES[indirect['direct']]), # TODO can't think of any....
        #                " before %s" % (indirect['phrase']), # TODO can't think of any....
                    
    if ncomp =='first':
        for rel in NOVEL_COMPS[ncomp]:
            phrases = [
                "%s first" % PP[rel],
                "first thing they %s" % PAST[rel]
            ]
            for phrase in phrases:
                if question.find(phrase) != -1:
                    cats.add(rel)
        
        # for eveything in before comp, look for phrae
        #"the object they were %s first%s" % (g.PP[r], indirect['phrase']),
        #            "the first thing they %s%s" % (PAST[r], indirect['phrase']), # TODO: can add with past tense later
         # can simplfy. "firt hing they %s" and "%s first"  
    if ncomp == 'longer':
        print()
        print("WILL BE more complicated than i thought b/c have to connect to object.... but it might be in indirect object ref... ")
        print("Indirect longer reference b/c in object ref")
        print(q_id, question)
        print()
        #for rel in NOVEL_COMPS[ncomp]:
        
        #    phrases = [
        #        "%s first" % PP[rel],
        #        "first thing they %s" % PAST[r]
        #    ]
        #    for phrase in phrases:
        #        if question.find(phrase) != -1:
        #            cats.add(act)
        
        #phrases = [
        #    "doing the action they did the longest",
        #    "doing the longest action",
        #    "doing the thing they spent the longest amount of time doing",
        #]
        # shoudlnt need this... b/c so
    if ncomp == 'repetition':
        print("Shouldn't have missed a repetition one", q_id, q['question'])
        
    if ncomp == 'objrel':
        # will be more complicated b/c need to connect refs with the words
        print("in obj rel", q_id, q['question'])
    return cats


def divideByComps(q, q_id, divided, ncomp):
    lst = NOVEL_COMPS[ncomp]
    
    # get list of dic of all ones in includes
    args = q['metrics']['direct_args'].split("-") + q['metrics']['direct_time'][1].split("-")
    cats = set()
    for arg in args:
        if arg == "":
            continue
        if arg in lst:
            cats.add(arg)
            
    if not cats:
        #print("cats is empty")
        cats = substringSearch(q_id, q, ncomp)
    # add q_id to divdeded (adding in arg to dic if needed)
    for arg in cats:
        if arg not in divided[ncomp]:
            divided[ncomp][arg] = []
        divided[ncomp][arg].append(q_id)
    # return
    return divided
    
def findWhichCompObjRel(q, q_id, divided, ncomp):
    lst = NOVEL_COMPS[ncomp]
    
    # get list of dic of all ones in includes
    args = q['metrics']['direct_args'].split("-") + q['metrics']['direct_time'][1].split("-")
    cats = set()
    objs = set()
    rels = set()
    
    for arg in args:
        if arg == "":
            continue
        tp = g.vType(arg)
        if tp == 'objects':
            objs.add(arg)
        if tp in ['attention', 'spatial', 'contact', 'verb']:
            rels.add(arg)
        
    
    if not objs or not rels:
        print("THis should not be happnening")
    
    for obj in objs:
        for rel in rels:
            if (obj, rel) in lst:
                cats.add((obj, rel))
                
    if not cats: 
        print('CATS IS EMPTY FOR THIS ONE')
            
    # add q_id to divdeded (adding in arg to dic if needed)
    for arg in cats:
        if arg not in divided[ncomp]:
            divided[ncomp][arg] = []
        divided[ncomp][arg].append(q_id)
    # return
    return divided
    

def splitNovelComp(d_path, group, balanced, ncomps):
    divided = {}
    
    for ncomp in ncomps:
        print(ncomp, len(ncomps[ncomp]))
        divided[ncomp] = {}
        count = 0
        
        QA = {}
        prev_vid = -1
        for q_id in ncomps[ncomp]:
            
            v_id = q_id[:5]
            if v_id != prev_vid:
                QA = getVideoQA(d_path, group, balanced, v_id)

            q = QA[q_id]
            #print(q_id, q['question'])
            
            if ncomp == 'objrel':
                divided = findWhichCompObjRel(q, q_id, divided, ncomp)
            else:
                divided = divideByComps(q, q_id, divided, ncomp)
                
            
            prev_vid = v_id
            if count % 10000 == 0:
                print(" "*4, count)
            count = count + 1
            
            if count == 100000:
                break
    return divided
                
    

In [ ]:
group = 'test'
novel_comp_tr_b = []
no_cat_tr_b = []
ranges = [(0, 100000), (100000, 200000), (200000, 400000), (400000, 600000), (600000, 900000), (900000, 1200000)]
        
for start, stop in ranges:
    novel_comp_tr_b.append(pickleLoad('../exports/%s/metrics/balanced/%s-%s-%s.txt' % (d_path, group, start, stop)))
    #with open('../exports/%s/metrics/balanced/%s-%s-%s.txt' % (d_path, group, start, stop), 'rb') as json_file:
    #    novel_comp_tr_b.append(json.load(json_file))
    no_cat_tr_b.append(pickleLoad('../exports/%s/metrics/balanced/%s-nocat-%s-%s.txt' % (d_path, group, start, stop)))
        

In [ ]:
for ncomp in divided_te_b:
    #if ncomp != 'before':
    #    continue
    print(ncomp)
    for spec in divided_te_b[ncomp]:
        print(" "*4, spec, len(divided_te_b[ncomp][spec]))
        
        #print(divided_te_b[ncomp][spec])

"YSKX3-466": {"question": "Before putting a pillow somewhere, did they touch the first thing they went behind?", "answer": "Yes", "attributes": {"type": "objRelExists", "structural": "verify", "semantic": "objrel", "ans_type": "binary", "video_id": "YSKX3"}, "global": "exists", "local": "r22-o4", "steps": 4, "id": "YSKX3-466", "metrics": {"novel_comp": [false, true, false, false, false], "indirects": [true, false, false, true], "direct_args": "r22-o4", "direct_time": ["before", "c077"]}}, 


 "T5ECU-2002": {"question": "Before starting to hold a vacuum, did they lean on a chair?", "answer": "No", "attributes": {"type": "objRelExists", "structural": "verify", "semantic": "objrel", "ans_type": "binary", "video_id": "T5ECU"}, "global": "exists", "local": "r16-o8", "steps": 3, "id": "T5ECU-2002", "metrics": {"novel_comp": [false, false, false, false, true], "indirects": [false, false, false, true], "direct_args": "r16-o8", "direct_time": ["before", "c137"]}}, 

In [ ]:
d_path = 'dataset'
ans_type_te_b = loadMetrics(ans_type_title, d_path, 'test', balanced=True)
ans_type_tr_b = loadMetrics(ans_type_title, d_path, 'train', balanced=True)
ans_type_te_a = loadMetrics(ans_type_title, d_path, 'test', balanced=False)
ans_type_tr_a = loadMetrics(ans_type_title, d_path, 'train', balanced=False)

In [ ]:
print()
print("TRAIN all")
for i in ans_type_tr_a:
    print(i, len(ans_type_tr_a[i]))
print()
print("TEST all")
for i in ans_type_te_a:
    print(i, len(ans_type_te_a[i]))
print()
print("TRAIN balanced")
for i in ans_type_tr_b:
    print(i, len(ans_type_tr_b[i]))
print()
print("TEST balanced")
for i in ans_type_te_b:
    print(i, len(ans_type_te_b[i]))

In [ ]:
d_path = 'dataset'
balanced = 'balanced'
a = 'all'
global_te_b = loadMetrics(templ_title, d_path, 'test', balanced)
global_tr_b = loadMetrics(templ_title, d_path, 'train', balanced)
global_te_a = loadMetrics(templ_title, d_path, 'test', a)
global_tr_a = loadMetrics(templ_title, d_path, 'train', a)

In [ ]:
if False:
    print()
    print("TRAIN all")
    for i in global_tr_a:
        print(i, len(global_tr_a[i]))
    print()
    print("TEST all")
    for i in global_te_a:
        print(i, len(global_te_a[i]))
    print()
print("TOTAL all")
for i in global_te_a:
    #print(i, len(global_te_a[i]) + len(global_tr_a[i]))
    print(i, (global_te_a[i] + global_tr_a[i]) / 1000)

if False:    
    print()
    print("TRAIN balanced")
    for i in global_tr_b:
        print(i, len(global_tr_b[i]))
    print()
    print("TEST balanced")
    for i in global_te_b:
        print(i, len(global_te_b[i]))
print()
print("TOTAL balanced")
for i in global_te_a:
    #print(i, len(global_te_b[i]) + len(global_tr_b[i]))
    print(i, (global_te_b[i] + global_tr_b[i]) / 1000)
print()

In [ ]:
d_path = 'dataset'
same_direct_te_b = loadMetrics(same_direct_title, d_path, 'test', balanced=True)
same_direct_tr_b = loadMetrics(same_direct_title, d_path, 'train', balanced=True)
same_direct_te_a = loadMetrics(same_direct_title, d_path, 'test', balanced=False)
same_direct_tr_a = loadMetrics(same_direct_title, d_path, 'train', balanced=False)

In [ ]:
print()
print("TRAIN all")
#for i in same_direct_tr_a:
#    print(i, len(same_direct_tr_a[i]))
print()
print("TEST all")
#for i in same_direct_te_a:
#    print(i, len(same_direct_te_a[i]))
print()
print("TRAIN balanced")
for i in same_direct_tr_b:
    print(i, len(same_direct_tr_b[i]))
print()
print("TEST balanced")
for i in same_direct_te_b:
    print(i, len(same_direct_te_b[i]))

In [ ]:
def getIndirectRefSummary(dic):
    # its video id -> tempalte -> local -> direct args # all/direct/one/two. in one is object, relation, action, time
    # I want: # direct
    # average # indirect per direct
    # average # of all categories, so 1, relation, tim etc
    # SO i need: total # direct, total # all (and subtract direct to get avg indirect per direct)
    # total number for each one , two, etc
    # 
    
    nums = {
        'all': 0,
        'direct': 0,
        'one-time': 0,
        'one-object': 0,
        'one-relation': 0,
        'one-action': 0,
        'two': 0
    }
    
    for v_id in dic:
        for t_id in dic[v_id]:
            for direct in dic[v_id][t_id]:
                d = dic[v_id][t_id][direct]
                nums['all'] = nums['all'] + len(d['all'])
                nums['direct'] = nums['direct'] + len(d['direct'])
                nums['one-time'] = nums['one-time'] + len(d['one']['time'])
                nums['one-object'] = nums['one-object'] + len(d['one']['object'])
                nums['one-relation'] = nums['one-relation'] + len(d['one']['relation'])
                nums['one-action'] = nums['one-action'] + len(d['one']['action'])
                nums['two'] = nums['two'] + len(['two'])
    return nums

print("TRAIN all")
#ir_summary_tr_a = getIndirectRefSummary(same_direct_tr_a)
ir_summary_tr_b = getIndirectRefSummary(same_direct_tr_b)
#ir_summary_te_a = getIndirectRefSummary(same_direct_te_a)
ir_summary_te_b = getIndirectRefSummary(same_direct_te_b)

In [ ]:
print("TRAIN all")
#print(ir_summary_tr_a)
print()
print("TEST all")
#print(ir_summary_te_a)
print()
print("TRAIN bal")
print(ir_summary_tr_b)
print()
print("TEST bal")
print(ir_summary_te_b)
print()

In [ ]:
# find percent with 1 and % with 2

for i in ir_summary_te_b:
    num = ir_summary_te_b[i]
    print(i, round(num / ir_summary_te_b['all'] * 100, 3))

In [ ]:
(ir_summary_te_b['all'] - ir_summary_te_b['direct'])# / ir_summary_te_b['direct']

In [ ]:
g_tr

In [ ]:
#g_tr, t_tr = splitCompoStepsByGlobTempl('dataset', 'train', 'templ_balanced')
#g_te, t_te = splitCompoStepsByGlobTempl('dataset', 'test', 'templ_balanced')

In [ ]:
sp5_g_tr = findUniversalSplit(g_tr, 5)
sp5_t_tr = findUniversalSplit(t_tr, 5)
sp5_g_te = findUniversalSplit(g_te, 5)
sp5_t_te = findUniversalSplit(t_te, 5)

In [ ]:
#g_tr, t_tr = splitCompoStepsByGlobTempl('dataset', 'train', 'templ_balanced')
#g_te, t_te = splitCompoStepsByGlobTempl('dataset', 'test', 'templ_balanced')
sp_g_tr_idx, sp_g_tr, sp_g_tr_templ = findGlobalSplit(g_tr, t_tr, .5)
sp_g_te_idx, sp_g_te, sp_g_te_templ = findGlobalSplit(g_te, t_te, .5)
sp_t_tr_idx, sp_t_tr, _ = findGlobalSplit(t_tr, None, .5)
sp_t_te_idx, sp_t_te, _ = findGlobalSplit(t_te, None, .5)

In [ ]:
s = ''
for i in sp_g_tr_idx:
    new_str = '%s: %s, ' % (i, sp_g_tr_idx[i])
    s += new_str
    
s

In [ ]:
import seaborn as sns

In [ ]:
def alphabetize(z):
    sort_z = sorted(z, key=lambda x: x[0])
    x = []
    y = []
    for a, b in sort_z:
        x.append(a)
        y.append(b)
    
    return x, y
    

def visSplit(dic, split, title):
    # 1- distribution of templates in training/test
    z = []
    
    size = 8
    plt.subplots(figsize=(size, size * .75))
    
    for i in dic: 
        z.append((i, dic[i][split]))
        
    
    if split == 'less':
        plt.title(title + " Training")
        x, y = alphabetize(z)
        sns.barplot(y, x)
        plt.show()
    
    if split == 'more':
        plt.title(title + " Testing")
        x, y = alphabetize(z)
        sns.barplot(y, x)
        plt.show()
    
    # 2- proportion of each template in each
#visSplit(sp5_g_tr, "less", "Global Split By 5 -")
#visSplit(sp5_t_tr, "less", "Templ Split By 5 -")
#visSplit(sp5_g_te, "more", "Global Split By 5 -")
#visSplit(sp5_t_te, "more", "Templ Split By 5 -")

visSplit(sp_g_tr, "less", "Global Split-")
visSplit(sp_g_tr_templ, "less", "Global Split-")
visSplit(sp_g_te, "more", "Global Split -")
visSplit(sp_g_te_templ, "more", "Global Split -")

#visSplit(sp_t_tr, "less", "Templ Split-")
#visSplit(sp_t_te, "more", "Templ Split -")

# Old Stuff Below

In [ ]:
# generalized saving
# go thorugh all and save into new file
# create a dict of title --> file paths so that easier in 

to_save = {
    "length": [lengths_a, lengths_b],
    "global": [global_a, global_b],
    "answer_type": [answer_type_a, answer_type_b],
    "steps": [steps_a, steps_b],
    "same_direct": [same_direct_a, same_direct_b],
    "novel_compositions": [illegal_comp_a, illegal_comp_b]
}

num_videos = 500
balanced = "bal"
all_vid = 'all'
paths = {}

for name in to_save:
    path_a = "../exports/metrics/%s_%s_%s.txt" % (all_vid, num_videos, name)
    path_b = "../exports/metrics/%s_%s_%s.txt" % (balanced, num_videos, name)
    
    if to_save[name][0] is None or to_save[name][1] is None:
        print(name, " is None!")
    with open(path_a, 'w+') as outfile:
        json.dump(to_save[name][0], outfile)
    
    with open(path_b, 'w+') as outfile:
        json.dump(to_save[name][1], outfile)
    paths[name] = [path_a, path_b]
        
print(paths)

with open("../exports/metrics/paths.txt", 'w+') as outfile:
    json.dump(paths, outfile)
    



In [ ]:
# save them!

with open("../exports/metrics/bal_500_length.txt", 'w') as outfile:
    json.dump(length_gen, outfile)
    

with open("../exports/metrics/bal_500_glob.txt", 'w') as outfile:
    json.dump(glob, outfile)

In [ ]:
#ISSUES
# 1- some things don't have somehting with all direct, even objLast which should... unless it's tied!


# TODO:
# Check lengths make sense for all and add up!

In [ ]:
qaTotal()

printLens(length_gen, "Video Lengths")
printLens(glob, "Global Variables")
printLens(answer_type, "Answer Types")
printLens(steps, "Number compositional Steps")
printLens(same_direct, "Have same direct (may not work)")
printLens(illegal_comp, "Illegal Compositions")

In [ ]:
 #TODOOOOOO
    # need! to change from "touching"!